In [36]:
import os
import scenic

scenic_script = "./examples/carla/dependencyAnalysisTest4.scenic"
scenario = scenic.scenarioFromFile(scenic_script)

/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/simulators/carla/model.scenic:56: UserWarning: the "carla" package is not installed; will not be able to run dynamic simulations
  warnings.warn('the "carla" package is not installed; '


In [65]:
def cacheExprTreeNodes(attribute, nodeSet=None):
    """cache all the nodes of the input attribute's expression tree to the dictionary"""
    if nodeSet is None:
        nodeSet = set()
    nodeSet.add(attribute)
    if attribute._dependencies == ():
        return nodeSet
    for dep in attribute._dependencies:
        cacheExprTreeNodes(dep, nodeSet)
    return nodeSet

def cacheAttributes(scenario, attributeList):
    dictionary = {}
    dictionary['objAttributes_names'] = []
    dictionary['objAttributes'] = []
    
    # cache all object attributes
    for i in range(len(scenario.original_objects)):
        obj = scenario.original_objects[i]
        obj_name = 'obj'+str(i)
        dictionary[obj_name] = {}
        
        for attribute in attributeList:
            dictionary[obj_name][attribute] = {}
            dictionary[obj_name][attribute]['self'] = getattr(obj, attribute)
            dictionary[obj_name][attribute]['set'] = cacheExprTreeNodes(getattr(obj, attribute), None)
            dictionary[obj_name][attribute]['dependent_names'] = []
            dictionary[obj_name][attribute]['jointly_dependent_names'] = []
            dictionary[obj_name][attribute]['dependent_attributes'] = set()
            dictionary[obj_name][attribute]['jointly_dependent_attributes'] = set()
            dictionary['objAttributes_names'].append(obj_name+"_"+attribute)
            dictionary['objAttributes'].append(getattr(obj, attribute))
    return dictionary

def checkDependenceOnAnotherAttribute(intersection, attr1_name, attr2_name, dictionary):
    """ checks whether the two attr1 and attr2 are jointly dependent on an intermediate variable
    or is both dependent on another attribute. 
    Output:
    True, if both dependent on another attribute
    False, otherwise
    """
    [obj1_name, attr1] = attr1_name.split('_')
    attr1_dependent_attrs = dictionary[obj1_name][attr1]['dependent_attributes']
    [obj2_name, attr2] = attr2_name.split('_')
    attr2_dependent_attrs = dictionary[obj2_name][attr2]['dependent_attributes']
    
    objAttributes_names = dictionary['objAttributes_names'] 
    for attr_name in objAttributes_names:
        if attr_name == attr1_name or attr_name == attr2_name:
            continue
        else:
            [obj_name, attr] = attr_name.split('_')
            attr_obj = dictionary[obj_name][attr]['self']
            
            if attr_obj in intersection and not (attr_obj in attr1_dependent_attrs) \
                and not (attr_obj in attr2_dependent_attrs):
                cachedSet = dictionary[obj_name][attr]['set']
                if len(intersection - cachedSet) == 0:
                    # attr1 and attr2 are both dependent on attr_name
                    return True
    return False
        

def dependencyAnalysis(scenario, attributeList):
    dictionary = cacheAttributes(scenario, attributeList)
    dictionary['numberOfObjects'] = len(scenario.original_objects)
    objAttributes_names = dictionary['objAttributes_names']
    for i in range(len(objAttributes_names)):
        for j in range(len(objAttributes_names)):
            if i < j:
                attr1_name = objAttributes_names[i]
                attr2_name = objAttributes_names[j]
                [obj_name1, attr1] = attr1_name.split('_')
                [obj_name2, attr2] = attr2_name.split('_')
                attribute1 = dictionary[obj_name1][attr1]
                attribute2 = dictionary[obj_name2][attr2]
                attr1_obj = attribute1['self']
                attr2_obj = attribute2['self']
                
                set1 = attribute1['set']
                set2 = attribute2['set']
                intersection = set1.intersection(set2)
                
                if attr1_obj in intersection and not (attr1_obj in attribute2['dependent_attributes']):
                    # attr2_obj is dependent on attr1_obj
                    attribute2['dependent_names'].append(attr1_name)
                    attribute2['dependent_attributes'].add(attr1_obj)
                elif attr2_obj in intersection and not (attr2_obj in attribute1['dependent_attributes']):
                    # attr1_obj is dependent on attr2_obj
                    attribute1['dependent_names'].append(attr2_name)
                    attribute1['dependent_attributes'].add(attribute2['self'])
                elif len(intersection) > 0 and not (attr1_obj in intersection or attr2_obj in intersection) \
                    and not checkDependenceOnAnotherAttribute(intersection, attr1_name, attr2_name, dictionary):
                    # the two attributes are jointly dependent (i.e. share intermediate variable(s))
                    attribute1['jointly_dependent_names'].append(attr2_name)
                    attribute2['jointly_dependent_names'].append(attr1_name)
                    attribute1['jointly_dependent_attributes'].add(attribute2['self'])
                    attribute2['jointly_dependent_attributes'].add(attribute1['self'])
                else:
                    pass
    
    return dictionary

In [66]:
d = dependencyAnalysis(scenario, ['position', 'heading'])

In [67]:
obj_name = 'obj0'
attribute = 'position'
print(d[obj_name][attribute]['dependent_names'])
print(d[obj_name][attribute]['jointly_dependent_names'])

['obj0_heading']
['obj1_position']


In [45]:
"""
Issue1: ahead/behind, left/right of uses the same heading angle as the referenced
        (1) As a result, position & heading are jointly dependent
        ==> what if we do not allow joint dependency between position and heading?
        This assumes that we can decouple joint dependency between the two, if exists.
        Is this true? Yes
        ==> Limitation: if many there are many jointly dependent features all at once, it may not be feasible to solve
        
        (2) an obj can have its position be dependent on its heading because its heading is the same as the 
        heading of another object to which the obj is depedent
        ==> is this only an issue with ego? because the ordering of the objects 
        ==> ==> solution: just keep the original objects ordering

Issue2: my assumption that jointly dependent and dependent relationships are disjoint is wrong
        (e.g. dependencyAnalysisTest4.scenic)
        ==> it's not possible to capture such case since the attribute contains the intermediate variable
        ==> another ordering process needs to be done within jointly dependent features based on dependence relations

Issue3: Need to check the case when multiple attributes are dependent on another attributes
        (e.g. )
"""

"\nIssue1: ahead/behind, left/right of uses the same heading angle as the referenced\n        (1) As a result, position & heading are jointly dependent\n        ==> what if we do not allow joint dependency between position and heading?\n        This assumes that we can decouple joint dependency between the two, if exists.\n        Is this true? Yes\n        ==> Limitation: if many there are many jointly dependent features all at once, it may not be feasible to solve\n        \n        (2) an obj can have its position be dependent on its heading because its heading is the same as the \n        heading of another object to which the obj is depedent\n        ==> is this only an issue with ego? because the ordering of the objects \n        ==> ==> solution: just keep the original objects ordering\n\nIssue2: my assumption that jointly dependent and dependent relationships are disjoint is wrong\n        (e.g. dependencyAnalysisTest4.scenic)\n        ==> it's not possible to capture such case

In [50]:
d['objAttributes_names']

['obj0_position',
 'obj0_heading',
 'obj1_position',
 'obj1_heading',
 'obj2_position',
 'obj2_heading']